In [41]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
req = requests.get('https://www.imsdb.com/scripts/Matrix,-The.html')
soup = BeautifulSoup(req.content, 'lxml')

In [5]:
script = soup.find('pre')

In [157]:
script.text[:1000]

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tTHE MATRIX\n\n\n\n\t\t\t\t\tWritten by\n\n\t\t\t\tLarry and Andy Wachowski\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tApril 8, 1996\n\n\n\n\n\n\n\n\tFADE IN ON:\n\n\tCOMPUTER SCREEN\n\n\tSo close it has no boundaries.\n\n\tA blinking cursor pulses in the electric darkness like a\n\theart coursing with phosphorous light, burning beneath\n\tthe derma of black-neon glass.\n\n\tA PHONE begins to RING, we hear it as though we were \n\tmaking the call.  The cursor continues to throb,\n\trelentlessly patient, until --\n\n\t\t\t\t\tMAN (V.O.)\n\t\t\tHello?\n\n\tData now slashes across the screen, information flashing\n\tfaster than we read.\n\n\t\t\t\t\tSCREEN\n\t\t\tCall trans opt:  received.\n\t\t\t2-19-96  13:24:18  REC:Log>\n\n\t\t\t\t\tWOMAN (V.O.)\n\t\t\tI'm inside.  Anything to report?\n\n\tWe listen to the phone conversation as though we were on\n\ta third line.  The man's name is CYPHER.  The woman, \n\tTRINITY.\n\n\t\

names are preceded by 2 \n\n\t\t\t\t\t, dialogue is preceded by \n\t\t\t, stage direction is preceded by \n\n\t.  Problems that immediately come to mind: 
- making sure preceding spoken line is dialogue by someone else and not some transition
- making sure the format is the same between all scripts

With the neat formatting, that's all I can think of.

Leap of faith: conditions listed are going to apply to other scripts.  Unlikely, but let's see how it goes.

In [156]:
possible_characters = set([re.sub('[\n\t]', '', tagged.text) for tagged in script.findAll('b') if 'INT.' not in tagged.text and 'EXT.' not in tagged.text and ':' not in tagged.text])

In [114]:
all_lines = [chunk.split('\n\t\t\t') for chunk in script.text.split('\n\n\t\t\t\t\t')]

In [122]:
all_lines[101]

['NEO',
 'Shitshitshit.\n\n\n\tEXT.  SKYSCRAPER\n\n\tThe downtown office of CorTechs, a software development\n\tcompany.\n\n\n\tINT.  CORTECHS OFFICE\n\n\tThe main offices are along each wall, the windows\n\toverlooking downtown Chicago.\n\n\tRHINEHEART, the ultimate company man, lectures Neo\n\twithout looking at him, typing at his computer\n\tcontinuously.\n\n\tNeo stares at two window cleaners on a scaffolding\n\toutside, dragging their rubber squeegees down across the\n\tsurface of the glass.']

In [160]:
void_directions = lambda line: re.sub('\n\n+\t.*$', '', line, flags=re.S)
response = [(lines[0], ' '.join([void_directions(line) for line in lines[1:]])) for lines in all_lines if 'NEO' in lines[0]]

In [161]:
neo_lines_index = [index - 1 for index, lines in enumerate(all_lines) if 'NEO' in lines[0]]

In [178]:
probable_characters = [(character, len([line for line in all_lines if line[0]==character])) for character in possible_characters]
probable_characters = [character[0] for character in probable_characters if character[1] > 0]

In [163]:
prompts = [(lines[0], ' '.join([void_directions(line) for line in lines[1:]])) for index, lines in enumerate(all_lines) if index in neo_lines_index]

In [180]:
[(prompt, response) for prompt, response in zip(prompts, response) 
     if prompt[1] and response[1] 
     and prompt[0] != response[0] 
     and ':' not in ' '.join([prompt[1], response[1]])]

[(('VOICE (O.S.)', 'Hey, Tommy-boy!  You in there?'),
  ('NEO', 'What do you want, Anthony?')),
 (('ANTHONY',
   'I need your help, man.  Desperate. They got me, man.  The shackles of  fascism.'),
  ('NEO', 'You got the money this time?')),
 (('DUJOUR', "Why don't you come to the party with us?"),
  ('NEO', "I don't know.  I have to work tomorrow.")),
 (('DUJOUR', "Come on.  It'll be fun."),
  ('NEO', "Yeah, yeah.  Sure, I'll go.")),
 (('TRINITY', 'Hello, Neo.'), ('NEO', 'How did you know that --')),
 (('TRINITY',
   "I know a lot about you.  I've been wanting to meet you for some time."),
  ('NEO', 'Who are you?')),
 (('TRINITY', 'My name is Trinity.'),
  ('NEO',
   'Trinity?  The Trinity?  The Trinity that cracked the I.R.S. Kansas City D-Base?')),
 (('TRINITY', 'That was a long time ago.'), ('NEO', 'Gee-zus.')),
 (('TRINITY', 'What?'), ('NEO', 'I just thought... you were a guy.')),
 (('TRINITY', 'Most guys do.'),
  ('NEO', 'Do you want to go sorewhere and talk?')),
 (('TRINITY', "No

There is so much that will make this data garbage as far as naming it dialogue goes.  Movies will have a lot of stage direction that will influence the conversation.  Looking up a line doesn't guarantee that it's the same interaction by any means.  Right now, I'm just hoping to have grammar show up in the model, realizing how difficult it's going to be.  I don't want to cut off conversations just for the presence of stage direction, but they often signal change in conversation.  Maybe count line breaks in the future.

In [296]:
voidDirections = lambda line: re.sub('\n\n+\t.*$', '', line, flags=re.S)

def displayDialogue(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'lxml')
    script = soup.find('pre')
    all_lines = [chunk.split('\n\t\t\t') for chunk in script.text.split('\n\n\t\t\t\t\t')]

    possible_characters = set([re.sub('[\n\t]', '', tagged.text) for tagged in script.findAll('b') if 'INT.' not in tagged.text and 'EXT.' not in tagged.text and ':' not in tagged.text])
    probable_characters = [(character, len([line for line in all_lines if line[0]==character])) for character in possible_characters]
    probable_characters = [character[0] for character in probable_characters if character[1] > 10]

    dialogues = []
    for character in probable_characters:
        response = [(lines[0], ' '.join([voidDirections(line) for line in lines[1:]])) for lines in all_lines if lines[0] == character]
        char_indicies = [index - 1 for index, lines in enumerate(all_lines) if lines[0] == character]
        prompts = [(lines[0], ' '.join([voidDirections(line) for line in lines[1:]])) for index, lines in enumerate(all_lines) if index in char_indicies]
        char_dialogues = [(prompt[1], response[1]) for prompt, response in zip(prompts, response) 
                             if prompt[1] and response[1] 
                             and prompt[0] != response[0] 
                             and ':' not in ' '.join([prompt[1], response[1]])]
        dialogues.extend(char_dialogues)
    return dialogues

In [297]:
display_dialogue('https://www.imsdb.com/scripts/Matrix,-The.html')

[('Did you get anything from the room?',
  'Their next target.  The name is Neo.'),
 ("We'll need a search running.", "It's already begun."),
 ('No!', 'The others were lost.'),
 ("I'm sorry, Tank.",
  'There is a problem.  Reagan has failed to secure the hardware.'),
 ("Never send a human to do a machine's job.",
  "But if Reagan has failed, why haven't they pulled the plug?"),
 ('Perhaps we are asking the wrong questions.', "Or he doesn't know."),
 ('What were you doing?', "You don't know."),
 ('Know what?', "I think they're trying to save him."),
 ('There is no spoon.', 'Lower level --'),
 ('... help.', 'Only human...'),
 ("Sir!  Sir!  There was gunfire -- we've lost communication with the roof!",
  'Remain at your posts.'),
 ('But, sir -- the fire -- we should evacuate!',
  'You will do as you are ordered!'),
 ('The trace was completed.', 'We have their position.'),
 ('The extermination unit is in place.', 'Order the strike.'),
 ("You're the Oracle?",
  "Bingo.  I got to say I love 

In [210]:
display_dialogue('https://www.imsdb.com/scripts/Alien.html')

[['\n\n\n   "Alien", early draft, by Dan O\'Bannon\n\n\n\n   \n\n\n\n\n                               ALIEN\n\n\n                  (project formerly titled STARBEAST)\n\n\n                Story by Dan O\'Bannon & Ronald Shusett\n\n\n                      Screenplay by Dan O\'Bannon\n\n\n\n\n\n                                                                       1976\n\n\n\n\n\n\n                              SYNOPSIS\n\n     En route back to Earth from a far part of the galaxy, the crew of the\n     starship SNARK intercepts a transmission in an alien language,\n     originating from a nearby storm-shrouded planet.\n\n     Mankind has waited centuries to contact another form of intelligent\n     life in the universe -- they decide to land and investigate.  Their\n     search takes them to a wrecked alien spacecraft whose doors gape open\n     -- it is dead and abandoned.  Inside they find, among other strange\n     things, the skeleton of one of the unearthly space travellers.\n\n    

## Attempt 2

Well that's not going to generalize well apparently.

In [241]:
url = 'https://www.imsdb.com/scripts/Alien.html'

In [242]:
req = requests.get(url)
soup = BeautifulSoup(req.content, 'lxml')
script = soup.find('pre')

In [243]:
possible_characters = [re.sub('[\n\t]|\s{2,}', '', tagged.text) for tagged in script.findAll('b') 
                       if 'INT' not in tagged.text 
                       and 'EXT' not in tagged.text 
                       and ':' not in tagged.text]

In [246]:
probable_characters = [(character, len([line for line in possible_characters if line==character])) for character in set(possible_characters)]
probable_characters = [character[0] for character in probable_characters if character[1] > 5]

In [247]:
probable_characters

['BROUSSARD',
 'COMPUTER',
 'ROBY',
 "BROUSSARD (CONT'D)",
 'MELKONIS',
 "ROBY (CONT'D)",
 'STANDARD',
 'HUNTER',
 "STANDARD (CONT'D)",
 'FAUST']

In [284]:
str(script)

'<pre>\n\n\n   <title>"Alien", early draft, by Dan O\'Bannon</title>\n\n\n\n   <pre>\n\n\n\n\n                               <u>ALIEN</u>\n\n\n                  (project formerly titled <u>STARBEAST</u>)\n\n\n                Story by Dan O\'Bannon &amp; Ronald Shusett\n\n\n                      Screenplay by Dan O\'Bannon\n\n\n\n\n\n<b>                                                                       1976\n</b>\n\n\n\n\n\n<b>                              SYNOPSIS\n</b>\n     En route back to Earth from a far part of the galaxy, the crew of the\n     starship SNARK intercepts a transmission in an <u>alien language</u>,\n     originating from a nearby storm-shrouded planet.\n\n     Mankind has waited centuries to contact another form of intelligent\n     life in the universe -- they decide to land and investigate.  Their\n     search takes them to a wrecked alien spacecraft whose doors gape open\n     -- it is dead and abandoned.  Inside they find, among other strange\n     things, 

In [294]:
re.findall('\ *ROBY.*(\n.+\n*.*)', script.text)

['\n          Executive Officer.......Cautious but intelligent -- a survivor.\n\n     DELL BROUSSARD,',
 '\n               Oh... God... am I cold... \n\n                              BROUSSARD',
 '\n               I feel like shit... \n\n                              BROUSSARD',
 '\n     seats.\n\n                              BROUSSARD',
 '\n                    (putting down the cat)\n               Cattle ranch!',
 '\n               Right.  Fire up all systems.\n\n     They begin to throw switches, lighting up their consoles.  The control',
 '\n               Where are we?\n\n                              STANDARD',
 '\n               What the hell?\n\n     Standard picks up a microphone.',
 "\n               Chaz, I've got something here on my\n               security alert.  A high priority from",
 '\n                    (punches buttons)\n               Computer, you have signalled a',
 '\n               What?  Why?\n\n                              COMPUTER',
 '\n               Un

In [269]:
print(re.sub('\s{2,}', '', str(script)))

<pre><title>"Alien", early draft, by Dan O'Bannon</title><pre><u>ALIEN</u>(project formerly titled <u>STARBEAST</u>)Story by Dan O'Bannon &amp; Ronald ShusettScreenplay by Dan O'Bannon<b>1976
</b><b>SYNOPSIS
</b>En route back to Earth from a far part of the galaxy, the crew of thestarship SNARK intercepts a transmission in an <u>alien language</u>,originating from a nearby storm-shrouded planet.Mankind has waited centuries to contact another form of intelligentlife in the universe -- they decide to land and investigate.Theirsearch takes them to a wrecked alien spacecraft whose doors gape open-- it is dead and abandoned.Inside they find, among other strangethings, the skeleton of one of the unearthly space travellers.Certain clues in the wrecked ship lead them across the hostile surfaceof the planet to a primitive stone pyramid, the only remnant of avanished civilization.Beneath this pyramid they find an ancient tombfull of fantastic artifacts.Lying dormant in the tomb are centuries-old

# New site

https://www.weeklyscript.com/ .  No random tabs and spaces to format here, so screw you imsdb. **Edit** Mein Gott 

Notes:
- remove (.*)'s


In [349]:
url = 'https://www.weeklyscript.com/Dead+Poets%20Society.html'

In [350]:
req = requests.get(url)
soup = BeautifulSoup(req.content, 'lxml')
script = soup.find('center')

In [371]:
script1 = re.sub('([A-Z]+\)*)\n', '\\1:\n', script.text)\
            .replace('\n\n', '#')\
            .replace('  ', ' ')

In [372]:
script1 = re.sub('\([a-z].*\)', '', script1).split('\n')

In [373]:
script1

['##',
 ';##Begin typing to search our script database:#or click here to view all writers and their scripts###',
 '  "In writing fiction, the more fantastic ',
 "  the tale, the plainer the prose should be. Don't ask your readers to ",
 '  admire your words when you want them to believe your story." - ',
 '  Ben Bova',
 ' [ more quotes ]#',
 '"DEAD POET\'S SOCIETY"#',
 'Screenplay by#',
 'Tom Schulman#',
 'Final Draft##',
 'INT. WELTON ACADAMY DINING HALL - DAY - VARIOUS SHOTS:#',
 'CREDITS ROLL:#',
 'On the left is a life-sized mural depicting a group of young #school boys looking up adoringly at a woman who represents #liberty. On the right is a mural showing young men gathered #around an industrialist in a corporate boardroom. Between #the murals stands a boy.#',
 'An odd, blaring MUSICAL SOUND starts and stops, interrupted #by the noise of pumping. A teacher hurries to the boy, adjusts #his tie, and leads him off.#',
 "On another wall is a full-sized portrait of a 19th century #Sco

In [374]:
def lineType(line):
    if re.search('[A-Z]:#$', line):
        return 'setting'
    if re.match('^[A-Z.,\']+:', line):
        return 'line'
    else:
        return 'scene'

In [375]:
lineType('INT. WELTON ACADAMY DINING HALL - DAY - VARIOUS SHOTS:#')

'setting'

In [376]:
lineType('He moves off, then stops in front of Charlie Dalton.#')

'scene'

In [377]:
[(line, lineType(line)) for line in script1]

[('##', 'scene'),
 (';##Begin typing to search our script database:#or click here to view all writers and their scripts###',
  'scene'),
 ('  "In writing fiction, the more fantastic ', 'scene'),
 ("  the tale, the plainer the prose should be. Don't ask your readers to ",
  'scene'),
 ('  admire your words when you want them to believe your story." - ',
  'scene'),
 ('  Ben Bova', 'scene'),
 (' [ more quotes ]#', 'scene'),
 ('"DEAD POET\'S SOCIETY"#', 'scene'),
 ('Screenplay by#', 'scene'),
 ('Tom Schulman#', 'scene'),
 ('Final Draft##', 'scene'),
 ('INT. WELTON ACADAMY DINING HALL - DAY - VARIOUS SHOTS:#', 'setting'),
 ('CREDITS ROLL:#', 'setting'),
 ('On the left is a life-sized mural depicting a group of young #school boys looking up adoringly at a woman who represents #liberty. On the right is a mural showing young men gathered #around an industrialist in a corporate boardroom. Between #the murals stands a boy.#',
  'scene'),
 ('An odd, blaring MUSICAL SOUND starts and stops, interr

## Download the scripts